In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors

In [2]:
z = tf.constant([1., 2., 3.])

In [3]:
scale = tfb.Scale(2.)

In [4]:
x = scale.forward(z)

In [5]:
x

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2., 4., 6.], dtype=float32)>

Bijectors are used to transform Tensor objects. Bijectors have methods to apply the forward as well as the inverse transformation.

In [6]:
scale.inverse(tf.constant([5., 3., 1.]))

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2.5, 1.5, 0.5], dtype=float32)>

Now, let's add a shift and a scaling transformation.

In [7]:
scale = tfb.Scale(2.)
shift = tfb.Shift(1.)

In [8]:
scale_and_shift = tfb.Chain([shift, scale])

In [10]:
scale_and_shift.forward(z)

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([3., 5., 7.], dtype=float32)>

We can chain several transformations to produce a more complex transformation. Any chain of smooth and invertible transformations will again be smooth and invertible.

When passing the bijectors objects to the constructor `tfb.Chain` the bijectors are applyed in reverse order.

In [11]:
scale_and_shift.inverse(tf.constant([2., 5., 8.]))

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.5, 2. , 3.5], dtype=float32)>

Bijectors can also be used to transform random variables and compute the log probability of events under the transformed distribution.

In [12]:
normal = tfd.Normal(loc=0, scale=1.)

In [13]:
z = normal.sample(3)
z

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.80663365, -1.2608837 , -2.3091867 ], dtype=float32)>

In [14]:
scale_and_shift = tfb.Chain([tfb.Shift(1.), tfb.Scale(2.)])

In [15]:
x = scale_and_shift.forward(z)

In [16]:
x

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.6132673, -1.5217674, -3.6183734], dtype=float32)>

Here x is just equal to z multiplied by the factor of 2 and shifted by 1.

In [18]:
log_prob_z = normal.log_prob(z)
log_prob_z

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-1.2442675, -1.7138524, -3.5851102], dtype=float32)>

z is a realization of a random variable and so has an associated log probability which we can compute with the log prob method of the distribution object. But x is also a realization of a random variable, corresponding to the transformed distribution defined by the scale and shift bijectors forward operation. And we want to evaluate the density of this transformed distribution at x. That is what the change of variables formula is used for.

In [20]:
log_prob_x = normal.log_prob(z) - scale_and_shift.forward_log_det_jacobian(z, event_ndims=0)

The log probability of x is given by the log probability of z minus the log of the jacobian determinant of the forward transformation. The computation of the log jacobian determinant is a key feature of bijectors and implemented as `forward_log_det_jacobian`. This method has two required arguments: the input tensor z as well as an `event_ndims` argument which specifies the number of event space dimensions present in the input tensor z.

Remember that z has shapes given by sample, batch and event shapes. The computation of the log jacobian determinant should be reduced over the event dimensions. But these shape properties cannot be directly extracted from z, which is why we need to explicitly pass the number of event dimensions with the `event_ndims` argument. 

In the example above the random variable z has empty batch and event dimensions and a one dimensional sample shape of length 3, so we pass `event_ndims`=0. The result is a tensor of log probability of length 3, one for each event in the sample.